In [ ]:
def find_leaders(statements):
    leaders = set()
    leaders.add(1)
    for i, stmt in enumerate(statements, 1):
        if 'GOTO' in stmt:
            target = int(''.join(filter(str.isdigit, stmt.split('GOTO')[1])))
            leaders.add(target)
            if i < len(statements):
                leaders.add(i + 1)

    return sorted(list(leaders))

def create_basic_blocks(statements, leaders):
    blocks = {}
    current_block = None
    block_count = 1

    for i, stmt in enumerate(statements, 1):
        if i in leaders:
            if current_block:
                block_count += 1
            current_block = f'B{block_count}'
            blocks[current_block] = {
                'statements': [],
                'predecessors': set(),
                'successors': set(),
                'dominators': set()
            }

        if current_block:
            blocks[current_block]['statements'].append(f"{i}. {stmt}")

    return blocks

def build_flow_graph(blocks):
    block_list = sorted(blocks.keys())

    for i, block in enumerate(block_list):
        last_stmt = blocks[block]['statements'][-1]

        if 'GOTO' in last_stmt:
            target = int(''.join(filter(str.isdigit, last_stmt.split('GOTO')[1])))
            for next_block in block_list:
                first_stmt_num = int(blocks[next_block]['statements'][0].split('.')[0])
                if first_stmt_num == target:
                    blocks[block]['successors'].add(next_block)
                    blocks[next_block]['predecessors'].add(block)

        if 'GOTO' not in last_stmt and i < len(block_list) - 1:
            next_block = block_list[i + 1]
            blocks[block]['successors'].add(next_block)
            blocks[next_block]['predecessors'].add(block)

def find_dominators(blocks):
    block_list = sorted(blocks.keys())

    for block in block_list:
        if block == 'B1':
            blocks[block]['dominators'] = {block}
        else:
            blocks[block]['dominators'] = set(block_list)

    changed = True
    while changed:
        changed = False
        for block in block_list[1:]:
            old_doms = blocks[block]['dominators'].copy()
            if blocks[block]['predecessors']:
                pred_doms = [blocks[pred]['dominators'] for pred in blocks[block]['predecessors']]
                blocks[block]['dominators'] = set.intersection(*pred_doms).union({block})
            if old_doms != blocks[block]['dominators']:
                changed = True

def find_natural_loops(blocks):
    loops = []
    for block in blocks:
        for succ in blocks[block]['successors']:
            if succ in blocks[block]['dominators']:
                loops.append(f"{block} -> {succ}")
    return loops

def main():
    statements = [
        "counter = 1",
        "sum = 0",
        "while counter <= 15 GOTO 7",
        "increment = counter * 3",
        "sum = sum + increment",
        "counter = counter + 1",
        "GOTO 3",
        "print(sum)"
    ]

    leaders = find_leaders(statements)
    print("Leader statements:")
    for leader in leaders:
        if leader-1 < len(statements):
            print(f"{leader}. {statements[leader-1].split(' L')[0]}")
        else:
            print(f"{leader}. [Target statement]")

    blocks = create_basic_blocks(statements, leaders)
    print("\nBasic blocks:")
    for block in sorted(blocks.keys()):
        stmt_nums = ' '.join(stmt.split('.')[0] for stmt in blocks[block]['statements'])
        print(f"{block}: contains: {stmt_nums}")

    build_flow_graph(blocks)
    print("\nProgram Flow Graph:")
    for block in sorted(blocks.keys()):
        for succ in sorted(blocks[block]['successors']):
            print(f"{block} -> {succ}")

    find_dominators(blocks)
    print("\nDominators:")
    for block in sorted(blocks.keys()):
        print(f"Dominators of {block}: {sorted(list(blocks[block]['dominators']))}")

    loops = find_natural_loops(blocks)
    print("\nNatural loops:")
    for loop in loops:
        print(loop)

if __name__ == "__main__":
    main()


Leader statements:
1. counter = 1
3. while counter <= 15 GOTO 7
4. increment = counter * 3
7. GOTO 3
8. print(sum)

Basic blocks:
B1: contains: 1 2
B2: contains: 3
B3: contains: 4 5 6
B4: contains: 7
B5: contains: 8

Program Flow Graph:
B1 -> B2
B2 -> B4
B3 -> B4
B4 -> B2

Dominators:
Dominators of B1: ['B1']
Dominators of B2: ['B1', 'B2']
Dominators of B3: ['B1', 'B2', 'B3', 'B4', 'B5']
Dominators of B4: ['B1', 'B2', 'B4']
Dominators of B5: ['B1', 'B2', 'B3', 'B4', 'B5']

Natural loops:
B3 -> B4
B4 -> B2


In [ ]:
statements = [
        "counter = 1",
        "sum = 0",
        "while counter <= 15 GOTO 7",
        "increment = counter * 3",
        "sum = sum + increment",
        "counter = counter + 1",
        "GOTO 3",
        "print(sum)"
    ]
def find_leaders(statements):
    leaders = set()
    # Rule 1: First instruction is a leader
    leaders.add(1)

    # Rule 2 & 3: Targets of jumps and statements following jumps
    for i, stmt in enumerate(statements, 1):
        if 'GOTO' in stmt:
            target = int(''.join(filter(str.isdigit, stmt.split('GOTO')[1])))
            leaders.add(target)
            print(target)
            if i < len(statements):
                leaders.add(i + 1)
                print(i+1)

    return sorted(list(leaders))
print(find_leaders(statements))

7
4
3
8
[1, 3, 4, 7, 8]
